# Querying MPC for objects in Allen Fields

## First load the pointing group info

In [1]:
import numpy as np
import pickle
from astropy.io.votable import parse
import matplotlib.pyplot as plt
import pandas as pd

import healpy as hp
import pointing_groups as pg

import requests
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time

from astropy.io import fits
from astropy.wcs import WCS
from astropy.wcs import NoConvergence

%matplotlib inline

In [2]:
with open('PickledPointings.pkl', 'rb') as f:
    Pointing_Groups = pickle.load(f)
    
den_visit,den_ccd,den_stellarnum = np.loadtxt('ref_data.txt',unpack=True)
unique_den_visit = np.unique(den_visit)
max_stellarnum = []
_= [ max_stellarnum.append(np.max(den_stellarnum[den_visit==visit])) for visit in unique_den_visit ]
max_stellarnum = np.array(max_stellarnum)

for i,group in enumerate(Pointing_Groups):
    group_max = np.concatenate([ (max_stellarnum[(unique_den_visit==x)]) for x in np.array(Pointing_Groups[i]["visit_id"],dtype=np.float_) ])
    if not(group_max.size):
        group_max=[-9999.]
    group_max = np.max(group_max)
    Pointing_Groups[i]['stellar_density'] = group_max
    
# visitToIndex is a dictionary that returns the index of a given visit
visitToIndex = {}
for i,pointing in enumerate(Pointing_Groups):
    for visit in pointing['visit_id']:
        visitToIndex[str(visit)] = i

In [3]:
Pointing_Groups[0].head(10)

visit_id                    date_obs          ra        dec  \
0    439116  b'2015-05-20 10:12:35.083'  257.655417 -29.255417   
1    439120  b'2015-05-20 10:17:14.409'  257.655333 -29.256250   
2    439124  b'2015-05-20 10:23:13.327'  257.657042 -29.256778   
3    439128  b'2015-05-20 10:29:16.399'  257.656083 -29.255667   
4    439707  b'2015-05-21 10:02:26.888'  257.652042 -29.258389   
5    439711  b'2015-05-21 10:07:16.391'  257.651875 -29.259500   
6    439715  b'2015-05-21 10:11:54.434'  257.652792 -29.259333   
7    439719  b'2015-05-21 10:16:36.741'  257.652708 -29.259056   
8    439723  b'2015-05-21 10:22:09.298'  257.653875 -29.259528   
9    440411  b'2015-05-22 09:57:57.005'  257.650625 -29.258944   

                                 filename  survey_night  stellar_density  
0  b'c4d_150520_101400_ooi_VR_v4.fits.fz'            20          40964.0  
1  b'c4d_150520_101841_ooi_VR_v4.fits.fz'            20          40964.0  
2  b'c4d_150520_102438_ooi_VR_v4.fits.fz'            20          40964.0  
3  b'c4d_150520_103042_ooi_VR_v4.fits.fz'            20          40964.0  
4  b'c4d_150521_100355_ooi_VR_v2.fits.fz'            21          40964.0  
5  b'c4d_150521_100844_ooi_VR_v2.fits.fz'            21          40964.0  
6  b'c4d_150521_101323_ooi_VR_v2.fits.fz'            21          40964.0  
7  b'c4d_150521_101806_ooi_VR_v2.fits.fz'            21          40964.0  
8  b'c4d_150521_102339_ooi_VR_v2.fits.fz'            21          40964.0  
9  b'c4d_150522_095923_ooi_VR_v2.fits.fz'            22          40964.0

## Set up tools for MPC querying

In [4]:
url     = 'https://www.minorplanetcenter.net/cgi-bin/mpcheck.cgi'
headers = {}

payload = {'year':'2018',
           'month':'01',
           'day': '16.67',
           'which':'obs',
           'ra':'',
           'decl':'',
           'TextArea':'',
           'radius':'90',
           'limit':'30.0',
           'oc':'500',
           'sort':'d',
           'mot':'h',
           'tmot':'s',
           'pdes':'u',
           'needed':'f',
           'ps':'n',
           'type':'1'
          }

In [5]:
def create_obs_table(df):
    """
    Create the input to the MPC post request.
    
    Input
    -----
    df : pandas dataframe
    The dataframe of observations for a particular request.
    
    Output
    ------
    textarea : string
    The string that should be the 'textarea' value in the payload dictionary
    """
    
    textarea = ''
    
    for idx in range(len(df)):
        c = SkyCoord(df['ra'].iloc[idx]*u.degree, df['dec'].iloc[idx]*u.degree)
        ra_hms = c.ra.hms
        dec_dms = c.dec.dms
        date_obs = df['date_obs'].iloc[idx].decode()#[2:-1]
        time_obj = Time(date_obs, format='iso', scale='utc')
        if dec_dms.d != 0:
            name = ("     %07i   %s %s %s.%s %02i %02i %06.3f%+03i %02i %05.2f                     W84\n" %
                        (df['visit_id'].iloc[idx], date_obs[:4], date_obs[5:7], 
                         date_obs[8:10], str(time_obj.mjd)[6:11],
                         ra_hms.h, ra_hms.m, ra_hms.s,
                         dec_dms.d, np.abs(dec_dms.m), np.abs(dec_dms.s)))
        else:
            if copysign(1, dec_dms.d) == -1.0:
                dec_dms_d = '-00'
            else:
                dec_dms_d = '+00'
            name = ("     %07i   %s %s %s.%s %02i %02i %06.3f%s %02i %05.2f                     W84\n" %
                        (df['visit_id'].iloc[idx], date_obs[:4], date_obs[5:7],
                         date_obs[8:10], str(time_obj.mjd)[6:11],
                         ra_hms.h, ra_hms.m, ra_hms.s,
                         dec_dms_d, np.abs(dec_dms.m), np.abs(dec_dms.s)))
        textarea += name
        
    return textarea

In [6]:
def parse_results(result):
    
    results_cut = result.text.split('\n')[12:-49]
    
    visit_id = []
    name = []
    ra_hour = []
    ra_min = []
    ra_sec = []
    dec_deg = []
    dec_min = []
    dec_sec = []
    v_mag = []
    ra_motion = []
    dec_motion = []
    
    for line in results_cut:
        #print(line)
        visit_id.append(int(line[6:12]))
        name.append(line[12:36])
        ra_hour.append(int(line[38:40]))
        ra_min.append(int(line[41:43]))
        ra_sec.append(float(line[44:48]))
        dec_deg.append(int(line[49:52]))
        dec_min.append(int(line[53:55]))
        dec_sec.append(int(line[56:58]))
        try:
            v_mag.append(float(line[60:64]))
        except ValueError:
            v_mag.append(-99.0)
        ra_motion.append('%s%i' % (line[84], int(line[82:84])))
        dec_motion.append('%s%i' % (line[91], int(line[89:91])))
            
    results_df = pd.DataFrame(np.array([visit_id, name, ra_hour, ra_min, ra_sec, 
                                       dec_deg, dec_min, dec_sec, v_mag, 
                                       ra_motion, dec_motion]).T, 
                             columns=['visit_id', 'name', 'ra_hour', 'ra_min', 'ra_sec', 
                                       'dec_deg', 'dec_min', 'dec_sec', 'v_mag', 
                                       'ra_motion', 'dec_motion'])
    
    results_df['visit_id'] = pd.to_numeric(results_df['visit_id'])
    results_df['ra_hour'] = pd.to_numeric(results_df['ra_hour'])
    results_df['ra_min'] = pd.to_numeric(results_df['ra_min'])
    results_df['ra_sec'] = pd.to_numeric(results_df['ra_sec'])
    results_df['dec_deg'] = pd.to_numeric(results_df['dec_deg'])
    results_df['dec_min'] = pd.to_numeric(results_df['dec_min'])
    results_df['dec_sec'] = pd.to_numeric(results_df['dec_sec'])
    results_df['v_mag'] = pd.to_numeric(results_df['v_mag'])
    results_df['ra_motion'] = pd.to_numeric(results_df['ra_motion'])
    results_df['dec_motion'] = pd.to_numeric(results_df['dec_motion'])
    
    return results_df

In [7]:
def run_mpc_requests(dataframe, field_label):
    
    """
    Inputs
    ------
    dataframe: pandas dataframe, pointing group dataframe
    
    field_label: str, identifier for field
    
    Returns
    -------
    results_df: pandas dataframe with MPC results for field
    """
    
    url     = 'https://www.minorplanetcenter.net/cgi-bin/mpcheck.cgi'
    headers = {}

    payload = {'year':'2018',
               'month':'01',
               'day': '16.67',
               'which':'obs',
               'ra':'',
               'decl':'',
               'TextArea':'',
               'radius':'90',
               'limit':'30.0',
               'oc':'500',
               'sort':'d',
               'mot':'h',
               'tmot':'s',
               'pdes':'u',
               'needed':'f',
               'ps':'n',
               'type':'1'
              }
    
    results_df = None

    ta = create_obs_table(dataframe)
    payload['TextArea'] = ta
    print(ta)
    res = requests.post(url, data=payload, headers=headers)

    if results_df is None:
        results_df = parse_results(res)
        results_df['field'] = field_label
    else:
        label_results_df = parse_results(res)
        label_results_df['field'] = field_label
        results_df = results_df.append(label_results_df, ignore_index=True)
            
    return results_df

## Pick a pointing group and get data with `run_mpc_requests`

This will query the online MPC database and get a dataframe with all MPC objects within 90 arcminutes of the center of each pointing.

In [192]:
pg_num = 302

In [193]:
results_df = run_mpc_requests(Pointing_Groups[pg_num], str(pg_num))

     0433934   2015 04 18.27595 13 28 54.360-07 22 49.00                     W84
     0433938   2015 04 18.27919 13 28 54.570-07 22 52.10                     W84
     0433942   2015 04 18.28244 13 28 54.620-07 22 50.10                     W84
     0433947   2015 04 18.29032 13 28 54.390-07 22 51.00                     W84
     0433951   2015 04 18.29355 13 28 54.880-07 22 50.10                     W84
     0434612   2015 04 19.26738 13 28 54.640-07 22 48.40                     W84
     0434616   2015 04 19.27062 13 28 54.840-07 22 52.00                     W84
     0434620   2015 04 19.27386 13 28 54.940-07 22 49.00                     W84
     0434624   2015 04 19.27711 13 28 54.990-07 22 50.20                     W84
     0434628   2015 04 19.28035 13 28 54.620-07 22 48.30                     W84
     0435500   2015 04 21.26838 13 28 54.350-07 23 02.20                     W84
     0435504   2015 04 21.27167 13 28 54.170-07 23 03.50                     W84
     0435508   2015 04 21.27

In [194]:
results_df

visit_id                      name  ra_hour  ra_min  ra_sec  dec_deg  \
0        433934   (229921) 1995 SC43            13      28    55.7       -7   
1        433934   (149701) 2004 HD43            13      29     8.9       -7   
2        433934            2015 GR55            13      29     6.6       -7   
3        433934            2007 XL60            13      28    53.3       -7   
4        433934   (479328) 2013 TY49            13      28    54.2       -7   
5        433934   (272180) 2005 OS31            13      29     2.4       -7   
6        433934    (74357) 1998 WZ12            13      28    56.1       -7   
7        433934            2007 TZ371           13      28    26.4       -7   
8        433934   (342502) 2008 UN176           13      28    40.2       -7   
9        433934   (205205) 2000 FA1             13      28    53.0       -7   
10       433934            2015 GH58            13      28    54.2       -7   
11       433934            2007 TN283           13      29    35.7       -7   
12       433934   (123025) 2000 SB279           13      29    25.8       -7   
13       433934            2017 XP10            13      29    42.0       -7   
14       433934            2007 TQ128           13      29    22.9       -7   
15       433934   (341500) 2007 TL399           13      29    44.2       -7   
16       433934   (506472) 2002 RJ272           13      28    47.1       -7   
17       433934   (474836) 2005 SA64            13      29    45.0       -7   
18       433934   (490235) 2008 WX34            13      28     6.4       -7   
19       433934    (10242) Wasserkuppe          13      28    24.8       -7   
20       433934   (238393) 2004 ET2             13      29    47.2       -7   
21       433934   (129907) 1999 TN78            13      29    37.3       -7   
22       433934     (9631) Hubertreeves         13      28    31.1       -7   
23       433934            2015 GG58            13      27    54.8       -7   
24       433934   (491595) 2012 SV58            13      29    57.4       -7   
25       433934   (153913) 2001 XO246           13      28    40.9       -7   
26       433934   (490443) 2009 SR146           13      29    40.9       -7   
27       433934            2000 FF8             13      28     7.7       -7   
28       433934   (389717) 2011 SH30            13      27    54.0       -7   
29       433934            2015 GJ58            13      29    37.1       -7   
...         ...                       ...      ...     ...     ...      ...   
10441    436192   (231878) 2000 UC77            13      24    14.9       -8   
10442    436192   (478576) 2012 TF81            13      33    47.8       -6   
10443    436192    (93659) 2000 UF104           13      23    13.2       -7   
10444    436192    (30286) 2000 HG61            13      33     9.5       -6   
10445    436192   (498681) 2008 ST242           13      33    31.3       -8   
10446    436192   (212987) 2009 DO16            13      30    50.6       -5   
10447    436192   (331804) 2003 RY15            13      34    44.6       -7   
10448    436192    (78026) 2002 JJ72            13      34    21.2       -6   
10449    436192   (181067) 2005 QG9             13      23    27.1       -7   
10450    436192            2005 SP145           13      32    47.9       -6   
10451    436192   (247281) 2001 SR155           13      24    55.5       -8   
10452    436192            2000 FG8             13      31    38.9       -8   
10453    436192   (434066) 2001 WO51            13      28    52.7       -8   
10454    436192            2006 RE116           13      26    16.9       -6   
10455    436192            2003 SL214           13      34    11.0       -6   
10456    436192    (34145) 2000 QV15            13      34    27.8       -6   
10457    436192            2017 VX11            13      34    51.7       -7   
10458    436192   (344307) 2001 UU130           13      32    19.9       -6   
10459    436192            2015 GV54            13      3

## Get pixel values and CCD Number

In [195]:
results_df['ccd'] = -99
results_df['x_pixel'] = -99
results_df['y_pixel'] = -99

In [196]:
results_df['total_motion'] = np.sqrt(results_df['ra_motion']**2. + results_df['dec_motion']**2.)

In [197]:
# Can comment this out if you don't want to select on slow movers (total_motion is arcsec/hr.)
res_df = results_df.query('total_motion < 10. & v_mag < 23.')
#res_df = results_df.query('v_mag < 19.')
res_df = res_df.reset_index(drop=True)

In [198]:
res_df.head(10)

visit_id                      name  ra_hour  ra_min  ra_sec  dec_deg  \
0    433934            2015 GS55            13      29    43.2       -6   
1    433934   (523790) 2015 HP9             13      29    38.3       -6   
2    433938            2015 GS55            13      29    43.2       -6   
3    433938   (523790) 2015 HP9             13      29    38.3       -6   
4    433942            2015 GS55            13      29    43.2       -6   
5    433942   (523790) 2015 HP9             13      29    38.2       -6   
6    433947            2015 GS55            13      29    43.1       -6   
7    433947   (523790) 2015 HP9             13      29    38.1       -6   
8    433951            2015 GS55            13      29    43.1       -6   
9    433951   (523790) 2015 HP9             13      29    38.1       -6   

   dec_min  dec_sec  v_mag  ra_motion  dec_motion field  ccd  x_pixel  \
0       37       38   22.8         -3           1   302  -99      -99   
1       13       15   21.6         -7           3   302  -99      -99   
2       37       38   22.8         -3           1   302  -99      -99   
3       13       15   21.6         -7           3   302  -99      -99   
4       37       38   22.8         -3           1   302  -99      -99   
5       13       15   21.6         -7           3   302  -99      -99   
6       37       38   22.8         -3           1   302  -99      -99   
7       13       14   21.6         -7           3   302  -99      -99   
8       37       37   22.8         -3           1   302  -99      -99   
9       13       14   21.6         -7           3   302  -99      -99   

   y_pixel  total_motion  
0      -99      3.162278  
1      -99      7.615773  
2      -99      3.162278  
3      -99      7.615773  
4      -99      3.162278  
5      -99      7.615773  
6      -99      3.162278  
7      -99      7.615773  
8      -99      3.162278  
9      -99      7.615773

In [223]:
data_loc = '/astro/store/epyc/users/smotherh/DECAM_Data_Reduction/warps/%i' % pg_num

In [235]:
data_loc = str('/astro/store/epyc/users/smotherh/DECAM_Data_Reduction/pointing_groups_hyak/' +
               'Pointing_Group_%i/processed_data/rerun/rerun_processed_data' % pg_num)

In [236]:
night_visits = np.unique(res_df['visit_id'])

In [238]:
for visit in night_visits:
    df_visit = res_df.query('visit_id == %i' % visit)
    print(data_loc)
    print(visit)
    for i in range(1, 63):
        #print(i)
        try:
            #print('%s/v%i/diffexp-%02i.fits' % (data_loc, visit, i))
            #hdulist = fits.open('%s/%02i/%i.fits' % (data_loc, i, visit))
            hdulist = fits.open('%s/0%i/calexp/calexp-0%i_%02i.fits' % (data_loc, visit, visit, i))
            w = WCS(hdulist[1].header)
        except FileNotFoundError:
            print(visit, i)
            continue
        for idx, obj_row in df_visit.iterrows():
            ra = '%i:%i:%.1f' % (obj_row['ra_hour'], obj_row['ra_min'], obj_row['ra_sec'])
            dec = '%i:%i:%.1f' % (obj_row['dec_deg'], obj_row['dec_min'], obj_row['dec_sec'])
            c = SkyCoord(ra, dec, frame='icrs', unit=(u.hourangle, u.deg))
            try:
                x_pix, y_pix = c.to_pixel(w)
                if (x_pix < 2010) and (x_pix > 0):
                    if (y_pix < 4100) and (y_pix > 0):
                        print(obj_row['name'], ra, dec)
                        print(x_pix, y_pix)
                        res_df['x_pixel'].iloc[idx] = x_pix
                        res_df['y_pixel'].iloc[idx] = y_pix
                        res_df['ccd'].iloc[idx] = i
            except NoConvergence:
                continue

/astro/store/epyc/users/smotherh/DECAM_Data_Reduction/pointing_groups_hyak/Pointing_Group_302/processed_data/rerun/rerun_processed_data
433934
433934 2
          2015 GS55      13:29:43.2 -6:37:38.0
842.2991045938447 2668.3756880185424


/epyc/opt/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


433934 61
/astro/store/epyc/users/smotherh/DECAM_Data_Reduction/pointing_groups_hyak/Pointing_Group_302/processed_data/rerun/rerun_processed_data
433938
433938 2
          2015 GS55      13:29:43.2 -6:37:38.0
830.5639054145329 2656.2304271884777
433938 61
/astro/store/epyc/users/smotherh/DECAM_Data_Reduction/pointing_groups_hyak/Pointing_Group_302/processed_data/rerun/rerun_processed_data
433942
433942 2
          2015 GS55      13:29:43.2 -6:37:38.0
838.189000450022 2653.264985722117
433942 61
/astro/store/epyc/users/smotherh/DECAM_Data_Reduction/pointing_groups_hyak/Pointing_Group_302/processed_data/rerun/rerun_processed_data
433947
433947 2
          2015 GS55      13:29:43.1 -6:37:38.0
834.7869582706093 2660.79398172292
433947 61
/astro/store/epyc/users/smotherh/DECAM_Data_Reduction/pointing_groups_hyak/Pointing_Group_302/processed_data/rerun/rerun_processed_data
433951
433951 2
          2015 GS55      13:29:43.1 -6:37:37.0
834.459561579478 2632.865603422859
433951 61
/astro/store

In [239]:
res_df.query('x_pixel > -1 and y_pixel > -1')

visit_id                      name  ra_hour  ra_min  ra_sec  dec_deg  \
0     433934            2015 GS55            13      29    43.2       -6   
2     433938            2015 GS55            13      29    43.2       -6   
4     433942            2015 GS55            13      29    43.2       -6   
6     433947            2015 GS55            13      29    43.1       -6   
8     433951            2015 GS55            13      29    43.1       -6   
10    434612            2015 GS55            13      29    38.3       -6   
12    434616            2015 GS55            13      29    38.3       -6   
14    434620            2015 GS55            13      29    38.3       -6   
16    434624            2015 GS55            13      29    38.3       -6   
18    434628            2015 GS55            13      29    38.2       -6   
20    435500            2015 GS55            13      29    28.4       -6   
22    435504            2015 GS55            13      29    28.4       -6   
24    435508            2015 GS55            13      29    28.4       -6   
26    435512            2015 GS55            13      29    28.4       -6   
28    435516            2015 GS55            13      29    28.4       -6   
30    436176            2015 GS55            13      29    23.5       -6   
32    436180            2015 GS55            13      29    23.5       -6   
34    436184            2015 GS55            13      29    23.5       -6   
36    436188            2015 GS55            13      29    23.5       -6   
38    436192            2015 GS55            13      29    23.5       -6   

    dec_min  dec_sec  v_mag  ra_motion  dec_motion field  ccd     x_pixel  \
0        37       38   22.8         -3           1   302    6  842.299105   
2        37       38   22.8         -3           1   302    6  830.563905   
4        37       38   22.8         -3           1   302    6  838.189000   
6        37       38   22.8         -3           1   302    6  834.786958   
8        37       37   22.8         -3           1   302    6  834.459562   
10       37        2   22.8         -3           1   302    6  708.159576   
12       37        2   22.8         -3           1   302    6  694.576426   
14       37        2   22.8         -3           1   302    6  706.128657   
16       37        1   22.8         -3           1   302    6  697.475816   
18       37        1   22.8         -3           1   302    6  704.823714   
20       35       49   22.8         -3           1   302    6  378.615811   
22       35       49   22.8         -3           1   302    6  373.732940   
24       35       49   22.8         -3           1   302    6  389.050238   
26       35       49   22.8         -3           1   302    6  375.178618   
28       35       49   22.8         -3           1   302    6  387.948768   
30       35       13   22.8         -3           1   302    6  238.620098   
32       35       13   22.8         -3           1   302    6  234.413041   
34       35       13   22.8         -3           1   302    6  229.439687   
36       35       13   22.8         -3           1   302    6  227.821069   
38       35       13   22.8         -3           1   302    6  237.428754   

        y_pixel  total_motion  
0   2668.375688      3.162278  
2   2656.230427      3.162278  
4   2653.264986      3.162278  
6   2660.793982      3.162278  
8   2632.865603      3.162278  
10  2374.601307      3.162278  
12  2363.535928      3.162278  
14  2357.564632      3.162278  
16  2355.225412      3.162278  
18  2370.503331      3.162278  
20  1830.370336      3.162278  
22  1840.306631      3.162278  
24  1834.259511      3.162278  
26  1820.547540      3.162278  
28  1814.705281      3.162278  
30  1502.931298      3.162278  
32  1510.238611      3.162278  
34  1503.091734      3.162278  
36  1506.482942      3.162278  
38  1491.790953      3.162278

In [178]:
res_df.query('ccd == 30')

Empty DataFrame
Columns: [visit_id, name, ra_hour, ra_min, ra_sec, dec_deg, dec_min, dec_sec, v_mag, ra_motion, dec_motion, field, ccd, x_pixel, y_pixel, total_motion]
Index: []

In [134]:
res_df.query('ccd == 26')

Empty DataFrame
Columns: [visit_id, name, ra_hour, ra_min, ra_sec, dec_deg, dec_min, dec_sec, v_mag, ra_motion, dec_motion, field, ccd, x_pixel, y_pixel, total_motion]
Index: []

In [240]:
res_df.query('name == "          2015 GS55     "')

visit_id                      name  ra_hour  ra_min  ra_sec  dec_deg  \
0     433934            2015 GS55            13      29    43.2       -6   
2     433938            2015 GS55            13      29    43.2       -6   
4     433942            2015 GS55            13      29    43.2       -6   
6     433947            2015 GS55            13      29    43.1       -6   
8     433951            2015 GS55            13      29    43.1       -6   
10    434612            2015 GS55            13      29    38.3       -6   
12    434616            2015 GS55            13      29    38.3       -6   
14    434620            2015 GS55            13      29    38.3       -6   
16    434624            2015 GS55            13      29    38.3       -6   
18    434628            2015 GS55            13      29    38.2       -6   
20    435500            2015 GS55            13      29    28.4       -6   
22    435504            2015 GS55            13      29    28.4       -6   
24    435508            2015 GS55            13      29    28.4       -6   
26    435512            2015 GS55            13      29    28.4       -6   
28    435516            2015 GS55            13      29    28.4       -6   
30    436176            2015 GS55            13      29    23.5       -6   
32    436180            2015 GS55            13      29    23.5       -6   
34    436184            2015 GS55            13      29    23.5       -6   
36    436188            2015 GS55            13      29    23.5       -6   
38    436192            2015 GS55            13      29    23.5       -6   

    dec_min  dec_sec  v_mag  ra_motion  dec_motion field  ccd     x_pixel  \
0        37       38   22.8         -3           1   302    6  842.299105   
2        37       38   22.8         -3           1   302    6  830.563905   
4        37       38   22.8         -3           1   302    6  838.189000   
6        37       38   22.8         -3           1   302    6  834.786958   
8        37       37   22.8         -3           1   302    6  834.459562   
10       37        2   22.8         -3           1   302    6  708.159576   
12       37        2   22.8         -3           1   302    6  694.576426   
14       37        2   22.8         -3           1   302    6  706.128657   
16       37        1   22.8         -3           1   302    6  697.475816   
18       37        1   22.8         -3           1   302    6  704.823714   
20       35       49   22.8         -3           1   302    6  378.615811   
22       35       49   22.8         -3           1   302    6  373.732940   
24       35       49   22.8         -3           1   302    6  389.050238   
26       35       49   22.8         -3           1   302    6  375.178618   
28       35       49   22.8         -3           1   302    6  387.948768   
30       35       13   22.8         -3           1   302    6  238.620098   
32       35       13   22.8         -3           1   302    6  234.413041   
34       35       13   22.8         -3           1   302    6  229.439687   
36       35       13   22.8         -3           1   302    6  227.821069   
38       35       13   22.8         -3           1   302    6  237.428754   

        y_pixel  total_motion  
0   2668.375688      3.162278  
2   2656.230427      3.162278  
4   2653.264986      3.162278  
6   2660.793982      3.162278  
8   2632.865603      3.162278  
10  2374.601307      3.162278  
12  2363.535928      3.162278  
14  2357.564632      3.162278  
16  2355.225412      3.162278  
18  2370.503331      3.162278  
20  1830.370336      3.162278  
22  1840.306631      3.162278  
24  1834.259511      3.162278  
26  1820.547540      3.162278  
28  1814.705281      3.162278  
30  1502.931298      3.162278  
32  1510.238611      3.162278  
34  1503.091734      3.162278  
36  1506.482942      3.162278  
38  1491.790953      3.162278

## Load images and create stamp mosaic

#### Create a dataframe with results from just one object

In [241]:
obj_df = res_df.query('name == "          2015 GS55     "')

#### Use the information in the dataframe to load the stamps

In [242]:
data_loc

'/astro/store/epyc/users/smotherh/DECAM_Data_Reduction/pointing_groups_hyak/Pointing_Group_302/processed_data/rerun/rerun_processed_data'

In [243]:
#data_loc = '/astro/store/epyc/users/smotherh/DECAM_Data_Reduction/imdiff/300/deepDiff'

In [246]:
data_loc = str('/astro/store/epyc/users/smotherh/DECAM_Data_Reduction/pointing_groups_hyak/' +
               'Pointing_Group_%i/processed_data/rerun/rerun_processed_data' % pg_num)

In [244]:
data_loc = '/astro/store/epyc/users/smotherh/DECAM_Data_Reduction/warps/%i' % pg_num

In [248]:
full_stamp_list = []
for idx in range(len(obj_df)):
    visit_id = obj_df.iloc[idx]['visit_id']
    ccd_id = obj_df.iloc[idx]['ccd']
    x_pix = obj_df.iloc[idx]['x_pixel']
    y_pix = obj_df.iloc[idx]['y_pixel']
    if ((x_pix == -99) | (y_pix == -99)):
        stamp_res = np.zeros((21,21))
    else:
        try:
            #print('%s/0%i/calexp/calexp-0%i_%02i.fits' % (data_loc, visit, visit, ccd_id))
            #hdulist = fits.open('%s/%02i/%i.fits' % (data_loc, ccd_id, visit_id))
            hdulist = fits.open('%s/0%i/calexp/calexp-0%i_%02i.fits' % (data_loc, visit_id, visit_id, ccd_id))
            #hdulist = fits.open('%s/v%i/diffexp-%02i.fits' % (data_loc, visit_id, ccd_id))
            w = WCS(hdulist[1].header)
        except FileNotFoundError:
            print(visit, i)
            stamp_res = np.zeros((21,21))
            continue
    print(x_pix, y_pix)
    coord_val = w.wcs_pix2world(np.array([[x_pix, y_pix]]), 1)
    c = SkyCoord(coord_val[0][0], coord_val[0][1], frame='icrs', unit=(u.deg, u.deg))
    print(c.ra.hms, c.dec.dms)
    print(obj_df.iloc[idx][['ra_hour', 'ra_min', 'ra_sec', 'dec_deg', 'dec_min', 'dec_sec']])
    stamp_res = hdulist[1].data[int(y_pix-10):int(y_pix+11),
                                int(x_pix-10):int(x_pix+11)]
        
    full_stamp_list.append(stamp_res)

842.2991045938447 2668.3756880185424
hms_tuple(h=13.0, m=29.0, s=43.18415042569285) dms_tuple(d=-6.0, m=-37.0, s=-37.71186361078904)
ra_hour      13
ra_min       29
ra_sec     43.2
dec_deg      -6
dec_min      37
dec_sec      38
Name: 0, dtype: object
830.5639054145329 2656.2304271884777
hms_tuple(h=13.0, m=29.0, s=43.183781901838216) dms_tuple(d=-6.0, m=-37.0, s=-37.71726539767542)
ra_hour      13
ra_min       29
ra_sec     43.2
dec_deg      -6
dec_min      37
dec_sec      38
Name: 2, dtype: object
838.189000450022 2653.264985722117
hms_tuple(h=13.0, m=29.0, s=43.18393932372814) dms_tuple(d=-6.0, m=-37.0, s=-37.719237535268064)
ra_hour      13
ra_min       29
ra_sec     43.2
dec_deg      -6
dec_min      37
dec_sec      38
Name: 4, dtype: object
834.7869582706093 2660.79398172292
hms_tuple(h=13.0, m=29.0, s=43.08341874955971) dms_tuple(d=-6.0, m=-37.0, s=-37.72130590568679)
ra_hour      13
ra_min       29
ra_sec     43.1
dec_deg      -6
dec_min      37
dec_sec      38
Name: 6, dtype: o

In [249]:
num_subplots = len(full_stamp_list)
x_subplots = np.ceil(np.sqrt(num_subplots))
y_subplots = np.ceil(np.sqrt(num_subplots))
fig = plt.figure(figsize=(4*x_subplots, 4*y_subplots))
for stamp_idx in range(num_subplots):
    fig.add_subplot(x_subplots, y_subplots, stamp_idx+1)
    plt.imshow(full_stamp_list[stamp_idx])

In [234]:
num_subplots = len(full_stamp_list)
x_subplots = np.ceil(np.sqrt(num_subplots))
y_subplots = np.ceil(np.sqrt(num_subplots))
fig = plt.figure(figsize=(4*x_subplots, 4*y_subplots))
for stamp_idx in range(num_subplots):
    fig.add_subplot(x_subplots, y_subplots, stamp_idx+1)
    plt.imshow(full_stamp_list[stamp_idx])